Run this notebook in the same virtual environment with superlinked server
to ensure the same version of `superlinked` framework.

In [1]:
!pip freeze | grep superlinked

superlinked==28.8.2
superlinked-server==1.34.0


In [ ]:
!pip install openai

In [2]:
from pathlib import Path
import sys

# depending on the user's setup
# we will try to find the superlinked_app directory
# and add it to the sys.path

cwd = Path.cwd()
if cwd.name == "superlinked-recipes":
    project_dir = cwd / "projects" / "recipe-search"
elif cwd.name == "notebooks":
    project_dir = cwd.parent
else:
    project_dir = cwd

superlinked_app_dir = project_dir / "superlinked_app"
assert superlinked_app_dir.exists(), (
    f"{superlinked_app_dir} does not exist\n"
    "are you sure you are in the recipe-search/notebooks directory?"
)

if str(project_dir) not in sys.path:
    sys.path.append(str(project_dir))
    print("project_dir is added to sys.path")
else:
    print("project_dir is already in sys.path")

project_dir is added to sys.path


In [3]:
from superlinked import framework as sl

from superlinked_app.index import index, recipe_schema
from superlinked_app.query import query
from superlinked_app.api import vector_database

from superlinked_app.search_cust import run_search  # Your new NLQ + query pipeline

import pandas as pd

/home/biso/development/my_projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValidationError: 1 validation error for Settings
app_id
  Extra inputs are not permitted [type=extra_forbidden, input_value='recipes', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

In [ ]:
source = sl.InteractiveSource(recipe_schema)
executor = sl.InteractiveExecutor(
    sources=[source],
    indices=[index],
    vector_database=vector_database,
)
app = executor.run()

21:07:43 httpx INFO   HTTP Request: GET https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333 "HTTP/1.1 200 OK"
21:07:43 httpx INFO   HTTP Request: GET https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/default/exists "HTTP/1.1 200 OK"
21:07:43 httpx INFO   HTTP Request: GET https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/default "HTTP/1.1 200 OK"
21:07:43 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/default/index?wait=true "HTTP/1.1 200 OK"
21:07:43 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/default/index?wait=true "HTTP/1.1 200 OK"
21:07:43 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/default/index?wait=true "HTTP/1.1 200 OK"
21:07

### The following examples are with custom openai base_url parameters
#### Setting openrouter and qwen-2.5-72b and it works

In [ ]:
params = {
    "cust_natural_query": "trofie from italian cuisine, to prepare a dinner and with preparation time lower than 45 minutes",
    "limit": 3,
}

# Use your run_search function to handle NLQ and querying
result = run_search(params["cust_natural_query"], app, limit=params.get("limit", 5))

# Print search parameters and convert results to pandas DataFrame
print(getattr(result.metadata, "search_params", "No search_params metadata"))
df = sl.PandasConverter.to_pandas(result)
df

21:07:48 httpx INFO   HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
Extracted parameters: {'name_query': 'trofie', 'ingredients_query': None, 'instructions_query': None, 'min_rating': None, 'max_rating': None, 'min_prep_time': None, 'max_prep_time': 45, 'min_cook_time': None, 'max_cook_time': None, 'max_calories': None, 'categories_include_all': ['dinner'], 'categories_include_any': None, 'categories_exclude': None, 'cuisines_include_all': ['Italian'], 'cuisines_include_any': None, 'cuisines_exclude': None, 'limit': None}
21:07:50 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.78it/s]

21:07:52 superlinked.framework.query.query_dag_evaluator INFO   evaluated query


21:07:52 httpx INFO   HTTP Request: POST https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/default/points/query "HTTP/1.1 200 OK"
21:07:52 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
{'similar_filter_TextSimilaritySpace_ec64_Recipe_Name_value_param__': 'trofie', 'similar_filter_TextSimilaritySpace_ec64_Recipe_Name_weight_param__': 1.0, 'hard_filter_Preparation_Time_be_less_than_or_equal_to_param__': 45.0, 'hard_filter_Category_contain_all_of_param__': ['Dinner'], 'hard_filter_Cuisine_contain_all_of_param__': ['Italian'], 'limit_param__': 3, 'select_param__': ['Name', 'Ingredient_Names_Text', 'Instructions', 'Rating_Value', 'Preparation_Time', 'Cooking_Time', 'Category', 'Cuisine', 'Calories', 'Carbohydrates', 'Cholesterol', 'Fiber', 'Protein', 'Saturated_Fat', 'Sodium', 'Sugar', 'Fat', 'Unsaturated_Fat', 'Nutrition', 'Ingredients', 'URL'], 'radius_param__': None, 'space_weight_TextSimilaritySpace_ec64_param__

,Name,Ingredient_Names_Text,Instructions,Rating_Value,Preparation_Time,Cooking_Time,Category,Cuisine,Calories,Carbohydrates,...,Saturated_Fat,Sodium,Sugar,Fat,Unsaturated_Fat,Nutrition,Ingredients,URL,id,similarity_score
0,Two-Ingredient Pizza Dough,self-rising_flour greek_yogurt spray,Gather all ingredients. Preheat the oven to 50...,4.6,15.0,10.0,[Dinner],[Italian],116.0,18.0,...,1.0,314.0,1.0,3.0,0.0,"{""Calories"": ""116 kcal"", ""Carbohydrates"": ""18 ...","[{'ingredient': 'self-rising flour', 'quantity...",https://www.allrecipes.com/recipe/244447/two-i...,rec136,0.766729
1,Trofie alla Genovese,pine_nuts basil parmesan_cheese extra-virgin_o...,Start by making the pesto: Heat a dry pan over...,0.0,20.0,40.0,[Dinner],[Italian],470.0,26.0,...,7.0,437.0,2.0,38.0,0.0,"{""Calories"": ""470 kcal"", ""Carbohydrates"": ""26 ...","[{'ingredient': 'pine nuts', 'quantity': '0.33...",https://www.allrecipes.com/recipe/284486/trofi...,rec217,0.597015


In [ ]:
rows = []

space_names = ["name", "ingredents", "instructions", "rating", "prep_time", "cooking_time", "calories", "protein"]

# order of spaces is the same as during index creation

for entry in result.entries:
    partial_scores = dict(zip(space_names, entry.metadata.partial_scores))
    row = {"id": entry.id, **partial_scores}
    rows.append(row)

df = pd.DataFrame(rows)
df

,id,name,ingredents,instructions,rating,prep_time,cooking_time,calories,protein
0,rec136,0.019882,0.0,0.0,0.248047,0.249268,0.249634,0.0,0.0
1,rec217,0.103882,0.0,0.0,0.000000,0.248535,0.244629,0.0,0.0


In [ ]:
params = {
    "cust_natural_query": "breakfast with eggs from German cuisine, with preparation time lower than 30 minutes with calories lower than 200",
    "limit": 3,
}


# Use your run_search function to handle NLQ and querying
result = run_search(params["cust_natural_query"], app, limit=params.get("limit", 5))

# Print search parameters and convert results to pandas DataFrame
print(getattr(result.metadata, "search_params", "No search_params metadata"))
df = sl.PandasConverter.to_pandas(result)
df

21:07:54 httpx INFO   HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
Extracted parameters: {'name_query': None, 'ingredients_query': 'eggs', 'instructions_query': None, 'min_rating': None, 'max_rating': None, 'min_prep_time': None, 'max_prep_time': 30, 'min_cook_time': None, 'max_cook_time': None, 'max_calories': 200, 'categories_include_all': ['breakfast'], 'categories_include_any': None, 'categories_exclude': None, 'cuisines_include_all': ['German'], 'cuisines_include_any': None, 'cuisines_exclude': None, 'limit': None}


Batches: 100%|██████████| 1/1 [00:00<00:00, 31.90it/s]

21:07:56 superlinked.framework.query.query_dag_evaluator INFO   evaluated query
21:07:56 httpx INFO   HTTP Request: POST https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/default/points/query "HTTP/1.1 200 OK"


21:07:57 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
{'similar_filter_TextSimilaritySpace_b35f_Recipe_Ingredient_Names_Text_value_param__': 'eggs', 'similar_filter_TextSimilaritySpace_b35f_Recipe_Ingredient_Names_Text_weight_param__': 1.0, 'hard_filter_Preparation_Time_be_less_than_or_equal_to_param__': 30.0, 'hard_filter_Calories_be_less_than_or_equal_to_param__': 200.0, 'hard_filter_Category_contain_all_of_param__': ['Breakfast'], 'limit_param__': 3, 'select_param__': ['Name', 'Ingredient_Names_Text', 'Instructions', 'Rating_Value', 'Preparation_Time', 'Cooking_Time', 'Category', 'Cuisine', 'Calories', 'Carbohydrates', 'Cholesterol', 'Fiber', 'Protein', 'Saturated_Fat', 'Sodium', 'Sugar', 'Fat', 'Unsaturated_Fat', 'Nutrition', 'Ingredients', 'URL'], 'radius_param__': None, 'space_weight_TextSimilaritySpace_ec64_param__': 1.0, 'space_weight_TextSimilaritySpace_b35f_param__': 1.0, 'space_weight_TextSimilaritySpace_2c3a_param__': 1.0, 'space_weight_Numb

,Name,Ingredient_Names_Text,Instructions,Rating_Value,Preparation_Time,Cooking_Time,Category,Cuisine,Calories,Carbohydrates,...,Saturated_Fat,Sodium,Sugar,Fat,Unsaturated_Fat,Nutrition,Ingredients,URL,id,similarity_score
0,Farm-Fresh Poached Eggs,water egg and_black_pepper_to_taste,Fill a wide pan with 4 cups water (about 3 inc...,5.0,5.0,10.0,[Breakfast],[American],72.0,0.0,...,2.0,84.0,0.0,5.0,0.0,"{""Calories"": ""72 kcal"", ""Carbohydrates"": ""0 g""...","[{'ingredient': 'water', 'quantity': '4', 'uni...",https://www.allrecipes.com/recipe/284908/farm-...,rec259,0.874213
1,French Toast Sticks,eggs powdered_sugar milk maple_syrup brown_sug...,"Gather all ingredients. Mix eggs, powdered sug...",4.7,10.0,10.0,[Breakfast],[American],167.0,30.0,...,1.0,182.0,16.0,3.0,0.0,"{""Calories"": ""167 kcal"", ""Carbohydrates"": ""30 ...","[{'ingredient': 'eggs', 'quantity': '2', 'unit...",https://www.allrecipes.com/recipe/280405/frenc...,rec148,0.850909
2,Cast Iron Skillet Vegetable Frittata,bacon_grease broccoli onion bell_pepper black_...,Preheat the oven to 350 degrees F (175 degrees...,4.5,20.0,30.0,[Breakfast],[American],173.0,10.0,...,4.0,355.0,4.0,10.0,0.0,"{""Calories"": ""173 kcal"", ""Carbohydrates"": ""10 ...","[{'ingredient': 'bacon grease', 'quantity': '3...",https://www.allrecipes.com/recipe/275456/cast-...,rec345,0.838125


In [ ]:
rows = []

space_names = ["name", "ingredents", "instructions", "rating", "prep_time", "cooking_time", "calories", "protein"]

# order of spaces is the same as during index creation

for entry in result.entries:
    partial_scores = dict(zip(space_names, entry.metadata.partial_scores))
    row = {"id": entry.id, **partial_scores}
    rows.append(row)

df = pd.DataFrame(rows)
df

,id,name,ingredents,instructions,rating,prep_time,cooking_time,calories,protein
0,rec259,0.0,0.124695,0.0,0.250000,0.250000,0.249634,0.0,0.0
1,rec148,0.0,0.102783,0.0,0.248901,0.249634,0.249634,0.0,0.0
2,rec345,0.0,0.095703,0.0,0.246826,0.248535,0.246826,0.0,0.0
